# Test LGL data
In which we format the LGL toponym data for processing.

In [55]:
from xml.dom import minidom
import pandas as pd

## Load data

In [3]:
lgl_file = '../../data/LGL/lgl.xml'
doc = minidom.parse(lgl_file)

In [66]:
# need (1) article title (2) text (3) toponyms
doc = minidom.parse(lgl_file)
get_node_val = lambda x: x[0].childNodes[0].nodeValue
topo_idx = ['start', 'end', 'tag', 'name', 'lat', 'lon']
art_data_combined = []
for art in doc.getElementsByTagName('article'):
    art_id = int(art.getAttribute('docid'))
    art_title = get_node_val(art.getElementsByTagName('title'))
    art_text = get_node_val(art.getElementsByTagName('text'))
    art_topos = art.getElementsByTagName('toponym')
    art_data = []
    for t in art_topos:
        t_start = int(get_node_val(t.getElementsByTagName('start')))
        t_end = int(get_node_val(t.getElementsByTagName('end')))
        t_tag = t.getElementsByTagName('gaztag')
        if(len(t_tag) > 0):
            t_id = int(t_tag[0].getAttribute('geonameid'))
        try:
            t_name = get_node_val(t.getElementsByTagName('name'))
            t_lat = get_node_val(t.getElementsByTagName('lat'))
            t_lon = get_node_val(t.getElementsByTagName('lon'))
            t_data = pd.Series([t_start, t_end, t_tag, t_name, t_lat, t_lon], index=topo_idx)
            art_data.append(t_data)
        except Exception as e:
#             print('skipping toponym %s in %d'%(art_id))
            pass
    if(len(art_data) > 0):
        art_data = pd.concat(art_data, axis=1).transpose()
        # add article etc.
        art_data.loc[:, 'doc_id'] = art_id
        art_data.loc[:, 'title'] = art_title
        art_data.loc[:, 'text'] = art_text
        art_data_combined.append(art_data)
art_data_combined = pd.concat(art_data_combined, axis=0)
print('%d unique topos'%(art_data_combined.shape[0]))

4462 unique topos


## GeoCorpora data
Let's figure out how to process GeoCorpora while we're at it.

In [91]:
import pandas as pd
geocorpora_file = '../../data/mined_tweets/GeoCorpora/geocorpora_1506879947339.tsv'
geocorpora_meta = '../../data/mined_tweets/GeoCorpora/geocorpora_rehydrated.tsv'
# geocorpora_raw = pd.read_csv(geocorpora_file, sep='\t', index_col=0, encoding='utf-8')
# geocorpora_raw.rename(columns={'geoNameId':'geonames_ID', 'text':'entity_string', 'char_position':'char_start', 'tweet_id_str':'docid', ''}, inplace=True)
# # compute char end
# geocorpora_raw.loc[:, 'char_end'] = geocorpora_raw.loc[:, 'char_start'] + geocorpora_raw.loc[:, 'text'].apply(len)
# # move topo data to places
# topo_keys = ['entity_string', 'latitude', 'longitude', 'char_start', 'char_end']
# geocorpora_raw.loc[:, 'places'] = geocorpora_raw.apply(lambda x: [{x.loc[k] for k in topo_keys}], axis=1)
# meta
geocorpora_meta = [l.strip().split('\t') for l in open(geocorpora_meta, 'r')]
geocorpora_meta = pd.DataFrame(geocorpora_meta[1:], columns=geocorpora_meta[0])
display(geocorpora_meta.head())

,text,id,favorite_count,place.name,coordinates,geo,lang,created_at,retweet_count,user.screen_name,user.id,user.description,user.location
0,Quake hits off California - A magnitude 5.7 ea...,560558919261892609,0,,,,en,Wed Jan 28 22:03:55 +0000 2015,0,RandallEToby,908652114,Professional Speaker / Entrepreneur / Trainer ...,NYC / NJ / Philly
1,RT @Farida_N: A dozen of #African organisation...,489092188048211968,0,,,,en,Tue Jul 15 17:00:39 +0000 2014,1,TakeOvaRecords,2166951330,TakeOvaRecords is looking for new talent Dj/pr...,Spain Madrid
2,RT @londil: Cover-up: Ukraine rebels destroyin...,490437952519286786,0,,,,en,Sat Jul 19 10:08:14 +0000 2014,14,tom_hanssen,782895128,,Heerlen
3,RT @moscow_ghost: It's happening #Syria Al Qae...,547136055812837377,0,,,,en,Mon Dec 22 21:06:15 +0000 2014,14,metesohtaoglu,46442765,"Journalist,researcher,(Global Jihadi Movements...",Istanbul - Turkiye
4,Will they protest the killing of police? - Lan...,547859448035672064,0,,,,en,Wed Dec 24 21:00:45 +0000 2014,1,TheKillingnews,535315178,It is all about The Killing,None


In [106]:
import json
meta_json_file = '../../data/mined_tweets/GeoCorpora/geocorpora_rehydrated.json'
meta_data = [json.loads(l.strip()) for l in open(meta_json_file, 'r')]
print(len(meta_data))
print(meta_data[0])

1751
{'contributors': None, 'truncated': False, 'text': 'Quake hits off California - A magnitude 5.7 earthquake struck off the coast of Northern California on Wednesday, a... http://t.co/RkcDyH1RCh', 'is_quote_status': False, 'in_reply_to_status_id': None, 'id': 560558919261892609, 'favorite_count': 0, 'source': '<a href="http://www.hootsuite.com" rel="nofollow">Hootsuite</a>', 'retweeted': False, 'coordinates': None, 'entities': {'symbols': [], 'user_mentions': [], 'hashtags': [], 'urls': [{'url': 'http://t.co/RkcDyH1RCh', 'indices': [118, 140], 'expanded_url': 'http://ow.ly/2TZxBz', 'display_url': 'ow.ly/2TZxBz'}]}, 'in_reply_to_screen_name': None, 'in_reply_to_user_id': None, 'retweet_count': 0, 'id_str': '560558919261892609', 'favorited': False, 'user': {'follow_request_sent': False, 'has_extended_profile': False, 'profile_use_background_image': True, 'default_profile_image': False, 'id': 908652114, 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/b

In [112]:
meta_cols = ['text', 'id', 'favorite_count', 'place.name', 'coordinates', 'geo', 'lang', 'created_at', 'retweet_count', 'user.screen_name', 'user.id', 'user.description', 'user.location']
def extract_meta(data, key):
    val = None
    if('.' in key):
        key_1, key_2 = key.split('.')
        if(data.get(key_1) is not None and data[key_1].get(key_2)):
            val = data[key_1][key_2]
    else:
        val = data[key]
    return val
meta_data_1 = [[extract_meta(x, y) for y in meta_cols] for x in meta_data]
meta_data_1 = pd.DataFrame(meta_data_1, columns=meta_cols)
print(meta_data_1.head())

                                                text                  id  \
0  Quake hits off California - A magnitude 5.7 ea...  560558919261892609   
1  RT @Farida_N: A dozen of #African organisation...  489092188048211968   
2  RT @londil: Cover-up: Ukraine rebels destroyin...  490437952519286786   
3  RT @moscow_ghost: It's happening\n#Syria Al Qa...  547136055812837377   
4  Will they protest the killing of police? - Lan...  547859448035672064   

   favorite_count place.name coordinates   geo lang  \
0               0       None        None  None   en   
1               0       None        None  None   en   
2               0       None        None  None   en   
3               0       None        None  None   en   
4               0       None        None  None   en   

                       created_at  retweet_count user.screen_name     user.id  \
0  Wed Jan 28 22:03:55 +0000 2015              0     RandallEToby   908652114   
1  Tue Jul 15 17:00:39 +0000 2014              1  